In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/')

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data=pd.read_csv('fake news/fake-news.zip/train.csv')

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
data.isna()

,id,title,author,text,label
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
20795,False,False,False,False,False
20796,False,False,False,False,False
20797,False,False,False,False,False
20798,False,False,False,False,False


In [ ]:
data.dropna(inplace=True)

In [ ]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# getting independent features
x=data.drop('label',axis=1) 
x

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
# dependent features
y=data['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [ ]:
x.shape,y.shape

((18285, 4), (18285,))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import  one_hot



## One Hot Encoding

In [ ]:
voc_size=5000
messages=x

In [ ]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
len(corpus)

18285

In [ ]:
one_hot_word=[one_hot(word,voc_size) for word in corpus]

In [ ]:
one_hot_word[18000]

[704, 4076, 3524, 4708, 2156, 3156, 1349, 648, 1302, 3996, 1190]

# **Model**

In [ ]:
sent_length=30
embedded_docs=pad_sequences(one_hot_word,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  240 2007 4404]
 [   0    0    0 ... 4892 3527 1190]
 [   0    0    0 ...  785 4596 4249]
 ...
 [   0    0    0 ...  506 4644 2286]
 [   0    0    0 ... 1644 4689 1430]
 [   0    0    0 ... 1353 4717 1894]]


In [ ]:

## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
embedded_docs.shape

(18285, 30)

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final

array([[   0,    0,    0, ...,  240, 2007, 4404],
       [   0,    0,    0, ..., 4892, 3527, 1190],
       [   0,    0,    0, ...,  785, 4596, 4249],
       ...,
       [   0,    0,    0, ...,  506, 4644, 2286],
       [   0,    0,    0, ..., 1644, 4689, 1430],
       [   0,    0,    0, ..., 1353, 4717, 1894]], dtype=int32)

In [ ]:
X_final.shape,y_final.shape

((18285, 30), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
192/192 [==============================] - 9s 46ms/step - loss: 0.3452 - accuracy: 0.8335 - val_loss: 0.2007 - val_accuracy: 0.9138
Epoch 2/50
192/192 [==============================] - 8s 44ms/step - loss: 0.1473 - accuracy: 0.9417 - val_loss: 0.1876 - val_accuracy: 0.9229
Epoch 3/50
192/192 [==============================] - 8s 44ms/step - loss: 0.1024 - accuracy: 0.9628 - val_loss: 0.1954 - val_accuracy: 0.9206
Epoch 4/50
192/192 [==============================] - 9s 45ms/step - loss: 0.0713 - accuracy: 0.9758 - val_loss: 0.2480 - val_accuracy: 0.9205
Epoch 5/50
192/192 [==============================] - 8s 44ms/step - loss: 0.0485 - accuracy: 0.9829 - val_loss: 0.2642 - val_accuracy: 0.9160
Epoch 6/50
192/192 [==============================] - 9s 45ms/step - loss: 0.0310 - accuracy: 0.9900 - val_loss: 0.3047 - val_accuracy: 0.9152
Epoch 7/50
192/192 [==============================] - 8s 44ms/step - loss: 0.0211 - accuracy: 0.9936 - val_loss: 0.3281 - val_accuracy: 0.9112

In [ ]:

from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)


Epoch 1/50
192/192 [==============================] - 9s 46ms/step - loss: 0.3502 - accuracy: 0.8384 - val_loss: 0.1978 - val_accuracy: 0.9153
Epoch 2/50
192/192 [==============================] - 8s 44ms/step - loss: 0.1501 - accuracy: 0.9416 - val_loss: 0.1886 - val_accuracy: 0.9213
Epoch 3/50
192/192 [==============================] - 8s 44ms/step - loss: 0.1096 - accuracy: 0.9578 - val_loss: 0.2056 - val_accuracy: 0.9239
Epoch 4/50
192/192 [==============================] - 9s 45ms/step - loss: 0.0851 - accuracy: 0.9678 - val_loss: 0.2291 - val_accuracy: 0.9191
Epoch 5/50
192/192 [==============================] - 9s 46ms/step - loss: 0.0683 - accuracy: 0.9749 - val_loss: 0.2371 - val_accuracy: 0.9180
Epoch 6/50
192/192 [==============================] - 9s 44ms/step - loss: 0.0563 - accuracy: 0.9804 - val_loss: 0.2818 - val_accuracy: 0.9168
Epoch 7/50
192/192 [==============================] - 8s 44ms/step - loss: 0.0422 - accuracy: 0.9854 - val_loss: 0.2965 - val_accuracy: 0.9176

In [ ]:

y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3146,  273],
       [ 282, 2334]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.908036454018227